In [37]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys
module_path = os.path.abspath(os.path.join('unlabeled_extrapolation'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from unlabeled_extrapolation.models import imnet_resnet
from unlabeled_extrapolation.models import imnet_models
from unlabeled_extrapolation.models import clip_model
from torch import nn
import torch
import torchvision
import numpy as np
import glob
import pickle
import unlabeled_extrapolation.utils.utils as utils
import json

from unlabeled_extrapolation.datasets import fmow
from unlabeled_extrapolation.baseline_train import preprocess_config
from unlabeled_extrapolation.extract_features import get_features_labels

import importlib
importlib.reload(imnet_resnet)
importlib.reload(imnet_models)


<module 'unlabeled_extrapolation.models.imnet_models' from '/juice/scr/ananya/cifar_experiments/unlabeled_extrapolation/unlabeled_extrapolation/models/imnet_models.py'>

In [6]:
import torch
torch.tensor(3.0).tolist()

3.0

# Get weight distance between resnets (e.g. full ft vs random vs lpft)

In [43]:
def get_param_weights_counts(net):
    weight_dict = {}
    count_dict = {}
    for param in net.named_parameters():
        name = param[0]
        weights = param[1]
        weight_dict[name] = weights
        count_dict[name] = np.prod(np.array(list(param[1].shape)))
    return weight_dict, count_dict

def get_l2_dist(weight_dict1, weight_dict2, count_dict, ignore='.fc.'):
    l2_dist = torch.tensor(0.0).cuda()
    count = 0.0
    for key in weight_dict1:
        if ignore not in key:
            l2_dist += torch.sum(torch.square(weight_dict1[key] - weight_dict2[key]))
            count += count_dict[key]
    l2_dist = l2_dist / count
    return float(l2_dist.cpu().detach().numpy())
            
def load_resnet50(config_path, checkpoint_path, clip=False):
    print(config_path)
    with open(config_path) as f:
        config = json.load(f)
    if clip:
        net = clip_model.ClipModel('RN50')
    else:
        net = imnet_resnet.ResNet50()
    net.new_last_layer(config['num_classes'])
    utils.load_ckp(checkpoint_path, net)
    net = net.cuda()
    return net

# Load datasets.
def load_test_dataset(config_path, idx, batch_size=64, num_workers=2):
    with open(config_path) as f:
        config = json.load(f)
    preprocess_config(config, config_path)
    test_config = config['test_datasets'][idx]
    if 'unlabeled_extrapolation' not in test_config['classname'] and 'torchvision' not in test_config['classname']:
        test_config['classname'] = 'unlabeled_extrapolation.' + test_config['classname']
#     if 'transforms' not in test_config:
#         test_config['transforms'] = config['default_test_transforms']
    test_data = utils.init_dataset(test_config)
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return test_data, test_loader

In [8]:
mocov2_model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar')
mocov2_model.cuda()

ResNet50(
  (_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [9]:
# Compare weight distance between checkpoints

def load_ckp(path, num_classes):
    net = imnet_resnet.ResNet50()
    net.new_last_layer(num_classes)
    utils.load_ckp(path, net)
    return net

model_path_fmts = {
    'ft_living17_best_path_fmt': '../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'ft_living17_samelr_path_fmt': '../logs/full_ft_living17_resnet50/optimizer.args.lr-0.0001_seed-{}_run{}/',
    'lpft_living17_path_fmt': '../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-{}_use_net_val_mode-True_run{}/',
}
checkpoint_ext = 'checkpoints/ckp_best_source_val_living'
config_ext = 'config.json'
num_classes = 17
distances = {
    'ft_living17_best_path_fmt': [],
    'ft_living17_samelr_path_fmt': [],
    'lpft_living17_path_fmt': [],

}


In [ ]:
wd, cd = get_param_weights_counts(mocov2_model)

for i in range(3):
    for key in model_path_fmts:
        ckp_path = model_path_fmts[key].format(i, i) + checkpoint_ext
        config_path = model_path_fmts[key].format(i, i) + config_ext
        net = load_resnet50(config_path, ckp_path)
        dist = get_l2_dist(wd, get_param_weights_counts(net)[0], cd)
        distances[key].append(dist)

In [61]:
for key in distances:
    print(key)
    print(np.mean(distances[key]))
    print(np.std(distances[key]) / np.sqrt(len(distances[key])) * 1.645)

ft_living17_best_path_fmt
2.2578324963736427e-06
6.152266444806382e-09
ft_living17_samelr_path_fmt
2.4158858498897945e-07
2.8288501226835223e-09
lpft_living17_path_fmt
6.773888827638075e-08
1.2904584535951586e-09


In [10]:
def add_features(id_val_loader, ood_val_loader, ood_features_dict, id_features_dict, name, net):
    id_features, _ = get_features_labels(net, id_val_loader)
    ood_features, _ = get_features_labels(net, ood_val_loader)
    id_features_dict[name] = id_features
    ood_features_dict[name] = ood_features

In [79]:
id_dists, ood_dists = {}, {}
for key in model_path_fmts:
    id_dists[key] = []
    ood_dists[key] = []
for i in range(3):
    ood_features, id_features = {}, {}
    for key in model_path_fmts:
        ckp_path = model_path_fmts[key].format(i, i) + checkpoint_ext
        config_path = model_path_fmts[key].format(i, i) + config_ext
        net = load_resnet50(config_path, ckp_path)
        print('loading data')
        _, id_val_loader = load_test_dataset(config_path, idx=0)
        _, ood_val_loader = load_test_dataset(config_path, idx=1)
        print('computing features')
        if 'mocov2' not in id_features:
            add_features(id_val_loader, ood_val_loader, ood_features, id_features, name='mocov2', net=mocov2_model)
        add_features(id_val_loader, ood_val_loader, ood_features, id_features, name=key, net=net)
        id_dists[key].append(np.mean(np.square(id_features[key] - id_features['mocov2'])))
        ood_dists[key].append(np.mean(np.square(ood_features[key] - ood_features['mocov2'])))
        
    
        
# For each model, get config path, checkpoint path
# Set model to val mode
# If iteration is 1, get features for mocov2 model
# Get features for other models

../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-0_run0/config.json
loading data
computing features
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.0001_seed-0_run0/config.json
loading data
computing features
../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-0_use_net_val_mode-True_run0/config.json
loading data
computing features
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-1_run1/config.json
loading data
computing features
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.0001_seed-1_run1/config.json
loading data
computing features
../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-1_use_net_val_mode-True_run1/config.json
loading data
computing features
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-2_run2/config.json
loading data
computing features
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.0001_seed-2_run2/config.json
loading data
computing features
../logs/lp_then_f

In [76]:
for key in model_path_fmts:
    id_dists[key].append(np.mean(np.square(id_features[key] - id_features['mocov2'])))
    ood_dists[key].append(np.mean(np.square(ood_features[key] - ood_features['mocov2'])))

In [95]:
for key in id_dists:
    print(key)
    print('{:.2f} ({:.2f})'.format(100*np.mean(id_dists[key]), 100*np.std(id_dists[key]) / np.sqrt(len(id_dists[key])) * 1.645))
    print('{:.2f} ({:.2f})'.format(100*np.mean(ood_dists[key]), 100*np.std(ood_dists[key]) / np.sqrt(len(ood_dists[key])) * 1.645))
#     print(str(np.mean(ood_dists[key])) + ' (' + str(np.std(ood_dists[key]) / np.sqrt(len(ood_dists[key])) * 1.645) + ')')
    

ft_living17_best_path_fmt
1.88 (0.01)
1.67 (0.01)
ft_living17_samelr_path_fmt
1.33 (0.02)
1.03 (0.01)
lpft_living17_path_fmt
0.11 (0.01)
0.09 (0.01)


In [81]:
ood_dists

{'ft_living17_best_path_fmt': [0.016827703, 0.016777152, 0.016641773],
 'ft_living17_samelr_path_fmt': [0.010265077, 0.010212453, 0.010457616],
 'lpft_living17_path_fmt': [0.00088527414, 0.00077893224, 0.0010297422]}

# Get feature distance and cosine distance across all datasets

In [38]:
mocov2_model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar')
mocov2_model.cuda()

mocotp_model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/mocotp_checkpoint_0200.pth.tar')
mocotp_model.cuda()

clip_resnet50 = clip_model.ClipModel('RN50')
clip_resnet50.cuda()

ClipModel(
  (_model): CLIP(
    (visual): ModifiedResNet(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), str

In [76]:
# Make dictionary of datasets, keep model path for each of them
# For each dataset, also keep the index of the ID and OOD set

config_ext = 'config.json'
living17_config = {
    'ft_fmt': '../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 0,
    'ood_idx': 1,
    'checkpoint_ext': 'checkpoints/ckp_best_source_val_living',
    'pretrained_net': mocov2_model,
}
entity30_config = {
    'ft_fmt': '../logs/full_ft_entity30_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_entity30_resnet50/optimizer.args.lr-3e-05_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 0,
    'ood_idx': 1,
    'checkpoint_ext': 'checkpoints/ckp_best_source_val_entity',
    'pretrained_net': mocov2_model,
}
cifar_stl_config = {
    'ft_fmt': '../logs/full_ft_cifar_stl_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_cifar_stl_resnet50/optimizer.args.lr-0.0001_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 0,
    'ood_idx': 1,
    'pretrained_net': mocov2_model,
    'checkpoint_ext': 'checkpoints/ckp_best_cifar10-test',
}
cifar_cifar_p1_config = {
    'ft_fmt': '../logs/full_ft_cifar_stl_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_cifar_stl_resnet50/optimizer.args.lr-0.0001_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 0,
    'ood_idx': 2,
    'pretrained_net': mocov2_model,
    'checkpoint_ext': 'checkpoints/ckp_best_cifar10-test',
}
domainnet_config = {
    'ft_fmt': '../logs/full_ft_domainnet_clip_resnet50/optimizer.args.lr-0.001_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_domainnet_clip_resnet50/optimizer.args.lr-3e-07_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 0,
    'ood_idx': 1,
    'pretrained_net': clip_resnet50,
    'checkpoint_ext': 'checkpoints/ckp_best_sketch_val',
}
fmow_config = {
    'ft_fmt': '../logs/full_ft_fmow_mocotp_fmow_resnet50/optimizer.args.lr-0.01_seed-{}_run{}/',
    'lpft_fmt': '../logs/lp_then_ft_valmode_fmow_mocotp_fmow_resnet50/optimizer.args.lr-0.0001_seed-{}_use_net_val_mode-True_run{}/',
    'id_idx': 2,
    'ood_idx': 1,
    'pretrained_net': mocotp_model,
    'checkpoint_ext': 'checkpoints/ckp_best_americas_val',
}
dataset_configs = {'living17': living17_config, 'cifar_stl': cifar_stl_config, 'entity30': entity30_config, 'domainnet_config': domainnet_config, 'fmow': fmow_config}


In [78]:
def cosine_distance(features1, features2):
    dot_prods = np.diag(features1 @ features2.T)
    norms1 = np.linalg.norm(features1, axis=-1)
    norms2 = np.linalg.norm(features2, axis=-1)
    assert(dot_prods.shape == norms1.shape == norms2.shape)
    return np.mean(1 - dot_prods / norms1 / norms2)

In [79]:
A = np.array([[1.0, 2.0], [2.0, 4.0], [-1.0, -2.0]])
B = np.array([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]])
cosine_distance(A, B)

[ 5. 10. -5.]
[0. 0. 2.]


0.6666666666666666

In [86]:

for dataset_name in dataset_configs:
    dataset_config = dataset_configs[dataset_name]
    id_dists, ood_dists = {}, {}
    id_cos_dists, ood_cos_dists = {}, {}
    for key in ['ft_fmt', 'lpft_fmt']:
        id_dists[key] = []
        ood_dists[key] = []
        id_cos_dists[key] = []
        ood_cos_dists[key] = []
    print(dataset_name)
    for i in range(3):
        print(i)
        ood_features, id_features = {}, {}
        for key in ['ft_fmt', 'lpft_fmt']:
            ckp_path = dataset_config[key].format(i, i) + dataset_config['checkpoint_ext']
            print(ckp_path)
            config_path = dataset_config[key].format(i, i) + config_ext
            if dataset_config['pretrained_net'] == clip_resnet50:
                net = load_resnet50(config_path, ckp_path, clip=True)
            else:
                net = load_resnet50(config_path, ckp_path)
            print('loading data')
            _, id_val_loader = load_test_dataset(config_path, idx=0)
            _, ood_val_loader = load_test_dataset(config_path, idx=1)
            print('computing features')
            if 'pretrained' not in id_features:
                add_features(id_val_loader, ood_val_loader, ood_features, id_features, name='pretrained',
                             net=dataset_config['pretrained_net'])
            add_features(id_val_loader, ood_val_loader, ood_features, id_features, name=key, net=net)
            id_dists[key].append(np.mean(np.square(id_features[key] - id_features['pretrained'])))
            ood_dists[key].append(np.mean(np.square(ood_features[key] - ood_features['pretrained'])))
            id_cos_dists[key].append(cosine_distance(id_features[key], id_features['pretrained']))
            ood_cos_dists[key].append(cosine_distance(ood_features[key], ood_features['pretrained']))
            print(id_cos_dists[key], ood_cos_dists[key])
    print(id_dists)
    print(ood_dists)
    for key in ['ft_fmt', 'lpft_fmt']:
        print(key)
        print('ID: {} ({})'.format(np.mean(id_dists[key])*100,
                                    np.std(id_dists[key]) / np.sqrt(3) * 1.645*100))
        print('OOD: {} ({})'.format(np.mean(ood_dists[key])*100,
                                    np.std(ood_dists[key]) / np.sqrt(3) * 1.645 * 100))
        print('ID Cos: {} ({})'.format(np.mean(id_cos_dists[key]),
                                    np.std(id_cos_dists[key]) / np.sqrt(3) * 1.645))
        print('OOD Cos: {} ({})'.format(np.mean(ood_cos_dists[key]),
                                    np.std(ood_cos_dists[key]) / np.sqrt(3) * 1.645))
    print('\n\n')

living17
0
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-0_run0/checkpoints/ckp_best_source_val_living
../logs/full_ft_living17_resnet50/optimizer.args.lr-0.001_seed-0_run0/config.json
loading data
computing features
[4.67558   3.9917853 6.6473618 ... 2.1182008 1.3549337 1.7400774]
[0.48317337 0.46502852 0.46414542 ... 0.5598278  0.6497544  0.5999994 ]
[3.395272  3.9816914 3.8054936 ... 2.0842257 3.66625   2.2855158]
[0.5447943  0.44049364 0.4638543  ... 0.68038917 0.5745655  0.5571817 ]
[0.4898652] [0.5106485]
../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-0_use_net_val_mode-True_run0/checkpoints/ckp_best_source_val_living
../logs/lp_then_ft_valmode_living17_resnet50/optimizer.args.lr-0.0001_seed-0_use_net_val_mode-True_run0/config.json
loading data
computing features
[4.209481  2.055533  6.4423857 ... 1.2113397 1.0543026 1.149074 ]
[0.05515689 0.05486029 0.01746386 ... 0.13552356 0.33409077 0.15171754]
[2.7399411 1.7807063 2.090605  ... 1.

loading data
computing features
[3.9319994 6.5176854 4.6102357 ... 5.5281205 1.9466087 2.7111235]
[0.4744051  0.5249852  0.4610446  ... 0.64300966 0.5272831  0.4823752 ]
[5.7871027 7.844076  4.2798643 ... 2.742321  4.1313076 2.832693 ]
[0.51213145 0.48287314 0.4943161  ... 0.5331961  0.49906963 0.46320784]
[0.49866283] [0.5079178]
../logs/lp_then_ft_valmode_entity30_resnet50/optimizer.args.lr-3e-05_seed-0_use_net_val_mode-True_run0/checkpoints/ckp_best_source_val_entity
../logs/lp_then_ft_valmode_entity30_resnet50/optimizer.args.lr-3e-05_seed-0_use_net_val_mode-True_run0/config.json
loading data
computing features
[1.4224004  3.9115226  1.6132871  ... 5.175416   0.59398645 0.9728582 ]
[0.03435159 0.01961905 0.03839356 ... 0.03727984 0.10046369 0.0821631 ]
[3.8094127  4.3104234  1.8363563  ... 0.91996217 1.1669114  0.8529795 ]
[0.01311731 0.02968073 0.02125788 ... 0.04045647 0.05770159 0.08957887]
[0.0638119] [0.06082605]
1
../logs/full_ft_entity30_resnet50/optimizer.args.lr-0.001_seed-

loading data
computing features
[ 7.652297  10.7482195 17.614935  ...  9.980169  12.409733  15.609694 ]
[0.57899964 0.42786235 0.3802929  ... 0.49470884 0.4949836  0.4155516 ]
[13.668944   9.476299  14.504084  12.238247  17.506996  12.665559
 12.689648   9.882077  13.56061   13.197294  16.414707  16.048874
 10.547228  12.396861  13.137909  10.668833  11.537601  20.440527
 11.759023  11.238415  10.416899  12.027033  12.743004  13.238935
 10.731982  10.442772   9.45111    9.090574  14.535299  12.45669
 11.314023  13.387826  11.967876  14.1621895 14.938705  15.456351
 12.672629  10.052034  10.940138  10.997204   8.163438   7.5506396
  8.925207   8.6129055  7.776163   9.04515    8.310535   9.5538845
  9.376056  10.8837385 12.481265  15.855584   5.9962754  9.004528
  5.039079   8.603579   9.66571   16.232338   8.712492   9.504068
 15.741575   6.6788177  7.3834505 16.863487   8.940628   5.9150076
  9.46736    8.899966   6.095837  11.657612  10.957411   6.86613
  8.707083   9.745555   9.49155

loading data
computing features
[ 6.065091 12.390602 14.475008 ... 11.595556 12.049736 13.391932]
[0.58310556 0.34786516 0.38089055 ... 0.5251256  0.54492235 0.4758423 ]
[13.573376  11.306246  12.616697  10.305402  16.85249   11.512687
 13.045239  11.328333  14.39358   11.527214  15.471645  13.057304
 11.589043  13.636433  12.495786  11.223455  12.440617  17.223703
 13.374347  13.586798  10.308365  14.716719  15.395834  16.089104
 11.734009  10.382088  10.378764   9.979741  14.208605  10.0907755
 10.257168  11.82902   11.155667  13.145938  15.478762  14.498886
 17.075241  11.697123  12.449148  11.272535   9.995357   8.34499
 10.860231   9.768131   9.401625   8.969673   9.002413  11.128576
  9.610123  11.732077  11.548084  12.029782   7.5051823 10.587029
  8.756509   9.599026   8.233203  15.970983   8.170543  10.87085
 12.951864   7.2931213  8.981699  13.727062   8.611952   7.5486875
  9.960283   9.778057   9.9436455 10.578586  10.393017   9.771946
 10.263511  13.926949  10.338837   9.7

loading data
computing features
[ 7.2282133 11.730301  15.650086  ... 10.75982   10.3352375 13.603532 ]
[0.56435174 0.3601125  0.42715293 ... 0.54607683 0.5927191  0.49617666]
[13.95858   10.605661  13.175467  11.129907  15.849309  13.653081
 11.22977   12.128241  11.274098  12.741585  16.635933  15.131783
 12.456398  10.609709  12.188841  12.000437  12.742583  22.52776
 12.817294  13.275377  11.080869  16.15225   13.281531  13.962912
 10.4225445 10.160112   9.105349   9.244533  13.0106125 10.008236
  9.384991  10.364969   9.96923   12.146717  15.445726  10.991623
 12.631055  11.728128  10.121108  11.992019   9.248308   9.610855
 11.709492  10.644308   9.307077   9.304434   9.870005  10.487091
 10.988893   9.769737  13.944389  13.512711   7.80935   10.436409
  7.614078   9.874156  11.094573  17.14987   10.0614605 10.079491
 14.696133   7.5260563  9.19145   14.019977   9.920814   8.793795
  9.531237   9.295196   7.1414757  9.527486   7.556288   8.803801
 10.472919  11.359797  10.134839 

In [83]:
id_cos_dists

{'ft_fmt': [], 'lpft_fmt': []}